In [3]:
from minio import Minio
from dotenv import load_dotenv
import os
import json

In [7]:
load_dotenv()

minio_client = Minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"), 
    secret_key=os.getenv("MINIO_SECRECT_KEY"), 
    secure=False 
)
bucket_name = os.getenv("MINIO_BUCKET_NAME")

In [20]:
import re

# Lista de bairros de Fortaleza/CE
bairros_fortaleza = [
    "Aerolândia", "Alagadiço", "Alto da Balança", "Alvaro Weyne", "Amadeu Furtado",
    "Ancuri", "Antônio Bezerra", "Autran Nunes", "Barra do Ceará", "Barroso",
    "Bela Vista", "Benfica", "Bonsucesso", "Bom Jardim", "Cais do Porto",
    "Cajazeiras", "Canindezinho", "Carlito Pamplona", "Cidade 2000", "Cidade dos Funcionários",
    "Coco", "Conjunto Ceará I", "Conjunto Ceará II", "Cristo Redentor", "Curió",
    "De Lourdes", "Demócrito Rocha", "Dendê", "Dionísio Torres", "Dunas",
    "Edson Queiroz", "Ellery", "Engenheiro Luciano Cavalcante", "Farias Brito", "Fátima",
    "Floresta", "Genibaú", "Granja Lisboa", "Granja Portugal", "Guajeru",
    "Icaraí", "Itaoca", "Itaperi", "Itaperi II", "Jabuti",
    "Jacarecanga", "Jangurussu", "Jardim América", "Jardim Iracema", "Jardim Guanabara",
    "João XXIII", "José Bonifácio", "José de Alencar", "Lagoa Redonda", "Luciano Cavalcante",
    "Manuel Sátiro", "Maraponga", "Meireles", "Mondubim", "Monte Castelo",
    "Montese", "Mucuripe", "Nossa Senhora das Graças", "Padre Andrade", "Parangaba",
    "Parque Araxá", "Parque Dois Irmãos", "Parque Iracema", "Parque Manibura", "Parque Presidente Vargas",
    "Passaré", "Paupina", "Pirambu", "Planalto Ayrton Senna", "Planalto Pici",
    "Presidente Kennedy", "Prefeito José Walter", "Quintino Cunha", "Sabiaguaba", "Sapiranga",
    "Serrinha", "São Bento", "São Gerardo", "São João do Tauape", "Tancredo Neves",
    "Varjota", "Vicente Pinzon", "Vila Ellery", "Vila Pery", "Vila União", "Vila Velha"
]

mensagens_com_bairros = []

In [18]:
import unicodedata
def normalizar_texto(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    ).lower()

Obj:  Portalnoticiasceara/2024-12-10/media/photo_5390.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5394.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5395.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5401.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5406.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5407.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5410.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5411.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5419.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5420.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5421.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5422.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5423.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5424.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5425.jpg.zstd
Obj:  Portalnoticiasceara/2024-12-10/media/photo_5429.j

In [19]:
bairros_normalizados = [normalizar_texto(bairro) for bairro in bairros_fortaleza]

NameError: name 'list_objects' is not defined

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
dados = []
list_objects = minio_client.list_objects(bucket_name, prefix='Portalnoticiasceara', recursive=True)
# Processar cada objeto encontrado
for obj in list_objects:
    if obj.object_name.endswith('.json'):  # Verificar se é um arquivo JSON
        print(f"Lendo o arquivo: {obj.object_name}")
        try:
            # Baixar o objeto
            response = minio_client.get_object(bucket_name=bucket_name, object_name=obj.object_name)
            
            # Ler o conteúdo
            data = response.read().decode('utf-8')
            
            # Parsear JSON
            json_data = json.loads(data)
            
            # Verificar se é uma lista
            if isinstance(json_data, list):
                for item in json_data:
                    # Extrair informações relevantes
                    message = item.get('message', '')
                    message_id = item.get('id', None)
                    date = item.get('date', None)
                    media_path = item.get('media_path', None)
                    
                    if message:
                        # Normalizar a mensagem
                        message_normalizada = normalizar_texto(message)
                        
                        # Identificar bairros mencionados
                        bairros_mencionados = [
                            bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                            if bairro_normalizado in message_normalizada
                        ]
                        
                        # Se algum bairro foi identificado, salvar os dados
                        if bairros_mencionados:
                            for bairro in bairros_mencionados:
                                dados.append({
                                    "id": message_id,
                                    "mensagem": message,
                                    "bairro": bairro,
                                    "data": date,
                                    "media_path": media_path
                                })
            
            # Fechar o response
            response.close()
            response.release_conn()
            
            with open('./silver_metadata.json', 'w', encoding='utf-8') as json_file:
                json.dump(dados, json_file, ensure_ascii=False, indent=4)

        except Exception as e:
            print(f"Erro ao processar {obj.object_name}: {e}")

In [ ]:
df = pd.DataFrame(dados)

In [ ]:
df.tail()

In [ ]:
df['bairro'].value_counts()

In [ ]:
import zstandard
decompressor = zstandard.ZstdDecompressor()

In [ ]:
def download_and_decompress(path, bucket_name):
    # Baixar o arquivo do MinIO
    result = minio_client.get_object(bucket_name, path)
    
    # Ler os dados comprimidos
    compressed_data = result.read()
    
    # Descomprimir os dados
    decompressor = zstandard.ZstdDecompressor()
    
    try:
        # Usando BytesIO para garantir que os dados sejam tratados como fluxo
        decompressed_data = decompressor.decompress(compressed_data)
        return decompressed_data
    except zstandard.ZstdError as e:
        print(f"Erro ao descomprimir o arquivo {path}: {e}")
        return None

In [ ]:
paths = df[df['id'] == 5777]['media_path']  # Carregar os paths do dataframe

for path in paths:
    print(f"Processando {path}...")
    decompressed_data = download_and_decompress(path, bucket_name)
    
    if decompressed_data:
        print(f"Arquivo {path} descomprimido com sucesso.")
        # Aqui você pode salvar ou processar os dados descomprimidos
    else:
        print(f"Falha ao descomprimir o arquivo {path}.")